In [1]:
import numpy as np
import pandas as pd

def generate_socathang_dataset(
    seed: int = 24012026,
    start_year: int = 2008,
    end_year: int = 2017
):
    """
    Sinh dữ liệu số ca nhập viện theo tháng (2008–2017)
    Phù hợp bài tập ARIMA & SARIMA:
    - Có xu thế
    - Có mùa vụ 12 tháng
    - KHÔNG dừng → cần sai phân
    """

    rng = np.random.default_rng(seed)

    # =========================
    # 1. Trục thời gian
    # =========================
    dates = pd.date_range(
        start=f"{start_year}-01-01",
        end=f"{end_year}-12-01",
        freq="MS"
    )
    n = len(dates)
    t = np.arange(n)

    # =========================
    # 2. Thành phần xu thế (trend)
    # =========================
    trend = 600 + 2.5 * t   # tăng dần theo năm → cần d = 1

    # =========================
    # 3. Thành phần mùa vụ 12 tháng
    # =========================
    month = dates.month.values
    seasonal = (
        180 * np.sin(2 * np.pi * month / 12) +
        90  * np.cos(2 * np.pi * month / 12)
    )  # mùa đông cao, mùa hè thấp → cần D = 1

    # =========================
    # 4. Nhiễu có tự tương quan (ARMA)
    # =========================
    eps = rng.normal(0, 120, size=n)
    arma = np.zeros(n)

    phi = 0.6     # AR(1)
    theta = -0.4  # MA(1)

    for i in range(1, n):
        arma[i] = phi * arma[i-1] + eps[i] + theta * eps[i-1]

    # =========================
    # 5. Tổng hợp chuỗi
    # =========================
    cases = trend + seasonal + arma

    # =========================
    # 6. Ràng buộc dữ liệu y tế
    # =========================
    cases = np.round(np.clip(cases, 50, None)).astype(int)

    # =========================
    # 7. Một vài bất thường nhỏ (để dạy tiền xử lý dữ liệu)
    # =========================
    cases[25] = int(cases[25] * 1.8)   # spike
    cases[78] = int(cases[78] * 0.6)   # dip

    # =========================
    # 8. Xuất DataFrame
    # =========================
    df = pd.DataFrame({
        "date": dates,
        "cases": cases
    })

    return df


if __name__ == "__main__":
    df = generate_socathang_dataset(seed=24012026)

    print(df.head())
    print(df.tail())
    print("Số quan sát:", len(df))

    df.to_csv(
        "socathang_simulated_2.csv",
        index=False,
        encoding="utf-8-sig"
    )

    print("✔ Đã lưu file: socathang_simulated_2.csv")


        date  cases
0 2008-01-01    768
1 2008-02-01   1055
2 2008-03-01    472
3 2008-04-01    579
4 2008-05-01    637
          date  cases
115 2017-08-01    628
116 2017-09-01    795
117 2017-10-01    779
118 2017-11-01    955
119 2017-12-01    972
Số quan sát: 120
✔ Đã lưu file: socathang_simulated_2.csv
